#### **Step 1: Install libraries**

In [ ]:
!pip install smolagents
!pip install smolagents[litellm]
!pip install gradio
!pip install pypdf
!pip install python-docx
!sudo apt install python3-gdbm

#### **Step 2: Clone repository**

In [ ]:
!git clone https://github.com/NSTiwari/AI-Agents-ML-Chandigarh.git

#### **Step 3: Configure HF Token**

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
GEMINI_API_KEY = userdata.get('gemini')

base_path = "/content/AI-Agents-ML-Chandigarh"

Login using HF CLI (optional)

In [ ]:
!huggingface-cli login

#### **Example 1a: Simple agent using open model**

In [ ]:
from smolagents import HfApiModel, CodeAgent

# Default model is Qwen/Qwen2.5-Coder-32B-Instruct.
model = HfApiModel()

agent = CodeAgent(
    tools=[],
    model=model,
)

agent.run("When was the last time India won the T20 World Cup? Against which team?")

#### **Example 1b: Simple agent with grounding using DuckDuckGo search engine**

In [ ]:
from smolagents import HfApiModel, CodeAgent, DuckDuckGoSearchTool

model = HfApiModel()

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
)

agent.run("When was the last time India won the T20 World Cup? Against which team?")

#### **Example 2: Agent that analyzes images with a proprietory model using LiteLLM**

In [ ]:
from PIL import Image
image = Image.open(f"{base_path}/image.jpg")
display(image)

In [ ]:
from smolagents import HfApiModel, CodeAgent, DuckDuckGoSearchTool, LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-exp",
    api_key = GEMINI_API_KEY
)

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
)

agent.run("Describe this image", images = [image])

#### **Example 3: Agent that solves your assignment, and submits it to your professor via an email**

*Note: You'll need to enable 2FA for your Gmail account and create a Google Apps password to send emails via SMTP.*

In [ ]:
from smolagents import HfApiModel, CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, tool

from pypdf import PdfReader
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


@tool
def read_pdf_file(file_path: str) -> str:
    """
    This function reads the first three pages of a PDF file and returns its content as a string.
    Args:
        file_path: The path to the PDF file.
    Returns:
        A string containing the content of the PDF file.
    """
    content = ""
    reader = PdfReader(f"{file_path}")
    print(len(reader.pages))
    pages = reader.pages[:3]
    for page in pages:
        content += page.extract_text()
    return content

@tool
def send_email(file_path: str, subject: str, body: str, receiver_email: str) -> str:
    """This function reads an input file and sends an email to the recipient with an attachment.
    Args:
        file_path: The path to the attachment.
        receiver_email: The receiver's email address.
        subject: The email subject.
        body: The email body.
    Returns:
        A text whether the email was sent successfully.
    """

    sender_email = userdata.get('EMAIL')
    sender_password = userdata.get('PASSWORD')

    # Create the message.
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body of the email.
    msg.attach(MIMEText(body, 'plain'))

    # Attach the file.
    try:
        with open(file_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)  # Encode file in base64
            part.add_header('Content-Disposition', f'attachment; filename={file_path.split("/")[-1]}')
            msg.attach(part)
    except Exception as e:
        print(f"Error attaching file: {e}")

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
            print("Email sent successfully")
    except Exception as e:
        print(f"Error: unable to send email. {e}")

    return "Email sent"


model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-exp",
    api_key = GEMINI_API_KEY
)


agent = CodeAgent(
    tools=[read_pdf_file, send_email],
    model=model,
    additional_authorized_imports=["pypdf", "docx"]
)


agent.run("""Please load the source file. This is an assignment that I need to submit by answering some questions below:
            1. What is the chemical formula of Magnesium Oxide?
            2. List the different daily life examples given in the PDF.
            3. Summarize what is the chapter about?
            Save the answers to these questions in a .docx file in current directory using the "docx" library.
            Additionally, write my details such as name {} roll number (501757), subject name, and submission date (today's date).
            Also, provide an undertaking confirming that all the work is original and done by me.
            Finally, send an email of the saved document as an attachment to {} using the send_email tool.""",

          additional_args=dict(source_file=f"{base_path}/Science_Class_X.pdf"))

#### **Example 4: Data Analyst that analyzes cricket matches (IPL)**

In [ ]:
from smolagents import HfApiModel, CodeAgent
model = HfApiModel()

agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=["numpy", "pandas", "matplotlib.pyplot", "seaborn", "os", "PIL"]
)


analysis = agent.run(
    """You are an expert Data Analyst.
Please load the source file and analyze its content and answer the questions below.
1. How many matches did Mumbai Indians win?
2. Which team won the match by highest number of runs? What was the date, and it was played between which two teams?
3. How many teams played in the tournament? List them.

Finally, make some relevant visualizations of different types (barchart, piechart, etc.) using matplotlib and save them in the current directory.
""",
    additional_args=dict(source_file=f"{base_path}/matches.csv"),
)

#### **Example 5: Image Generation by importing a Hugging Face Space as tool**

In [ ]:
from smolagents import CodeAgent, HfApiModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-dev",
    name="image_generator",
    description="Generate an image from a prompt"
)

image_generation_tool("A sunny beach")


model = HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct")
agent = CodeAgent(tools=[image_generation_tool],
                  model=model,
                  additional_authorized_imports=["io", "PIL"])

agent.run(
    "Improve this prompt, then generate its image and save a copy of it in the current directory as a .JPG file using PIL library.",
    additional_args={'user_prompt': 'Cherry Blossom in Japan.'}
)